In [1]:
!pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00


In [10]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!

In [9]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/reddit_preprocessed.csv')
df = df.drop('Unnamed: 0', axis=1)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [3]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0
    Uninstalling packaging-26.0:
      Successfully

In [11]:
print(df['clean_comment'].isnull().sum())

131


In [12]:
print(df.isnull().sum())

clean_comment    131
category           0
dtype: int64


In [13]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# --- STEP 1: Fix the NaN Error ---
# Drop any rows where clean_comment became null/nan after preprocessing
df = df.dropna()
# Double check to ensure no floats remain
df['clean_comment'] = df['clean_comment'].astype(str)

print(f"Data ready. Total rows: {len(df)}")

# --- STEP 2: Define Experiment Configurations ---
# We will loop through these combinations
experiment_configs = [
    {"type": "BoW",    "ngram": (1, 1), "vectorizer": CountVectorizer(max_features=10000, ngram_range=(1, 1))},
    {"type": "BoW",    "ngram": (1, 2), "vectorizer": CountVectorizer(max_features=10000, ngram_range=(1, 2))},
    {"type": "BoW",    "ngram": (1, 3), "vectorizer": CountVectorizer(max_features=10000, ngram_range=(1, 3))},
    {"type": "TF-IDF", "ngram": (1, 1), "vectorizer": TfidfVectorizer(max_features=10000, ngram_range=(1, 1))},
    {"type": "TF-IDF", "ngram": (1, 2), "vectorizer": TfidfVectorizer(max_features=10000, ngram_range=(1, 2))},
    {"type": "TF-IDF", "ngram": (1, 3), "vectorizer": TfidfVectorizer(max_features=10000, ngram_range=(1, 3))},
]

# Set the experiment name in MLflow
mlflow.set_experiment("EXP 2 - BOW vs TF-IDF")

# --- STEP 3: Run the Experiment Loop ---
for config in experiment_configs:
    run_name = f"{config['type']}_ngram_{config['ngram']}"
    print(f"\n🚀 Running: {run_name} ...")

    with mlflow.start_run(run_name=run_name):
        # 1. Log Parameters
        mlflow.log_param("vectorizer_type", config['type'])
        mlflow.log_param("ngram_range", str(config['ngram']))
        mlflow.log_param("max_features", 10000)

        # 2. Split Data (Using random_state=42 ensures the split is the SAME for every loop)
        X_train, X_test, y_train, y_test = train_test_split(
            df.clean_comment,
            df.category,
            test_size=0.2,
            random_state=42
        )

        # 3. Vectorize
        vectorizer = config['vectorizer']
        # Fit only on training data to prevent data leakage
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)

        # 4. Train Model
        # We use the same parameters as Experiment 1 for a fair comparison
        rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
        rf.fit(X_train_vec, y_train)

        # 5. Evaluate
        y_pred = rf.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)

        # 6. Log Metrics
        mlflow.log_metric("accuracy", accuracy)

        # Optional: Log Precision/Recall for each class
        report = classification_report(y_test, y_pred, output_dict=True)
        mlflow.log_metric("precision_macro", report['macro avg']['precision'])
        mlflow.log_metric("recall_macro", report['macro avg']['recall'])
        mlflow.log_metric("f1_macro", report['macro avg']['f1-score'])

        print(f"   ✅ Completed. Accuracy: {accuracy:.4f}")

print("\n🎉 All experiments finished!")

Data ready. Total rows: 36662


2026/02/13 17:29:20 INFO mlflow.tracking.fluent: Experiment with name 'EXP 2 - BOW vs TF-IDF' does not exist. Creating a new experiment.



🚀 Running: BoW_ngram_(1, 1) ...
   ✅ Completed. Accuracy: 0.6503
🏃 View run BoW_ngram_(1, 1) at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/2/runs/a25e6b5eb1274834ac73e09fa3e6a4cb
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/2

🚀 Running: BoW_ngram_(1, 2) ...
   ✅ Completed. Accuracy: 0.6468
🏃 View run BoW_ngram_(1, 2) at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/2/runs/1172c33675ec472a9f444dbc2318168b
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/2

🚀 Running: BoW_ngram_(1, 3) ...
   ✅ Completed. Accuracy: 0.6494
🏃 View run BoW_ngram_(1, 3) at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/2/runs/39d3c991bb6e41c8aa8213a8720a77d3
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/

In [7]:
df.shape

(6792, 3)

In [8]:
df.head()

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1.0
1,1,buddhism much lot compatible christianity espe...,1.0
2,2,seriously say thing first get complex explain ...,-1.0
3,3,learned want teach different focus goal not wr...,0.0
4,4,benefit may want read living buddha living chr...,1.0
